In [1]:
from qondense.tapering import tapering
from qondense.cs_vqe import cs_vqe
from qondense.utils.operator_toolkit import exact_gs_energy
import qondense.utils.qonversion_tools as qonvert
from qondense.utils.ordering_heuristics import ordering_heuristics
import json
import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf
from openfermion.circuits import ( uccsd_singlet_get_packed_amplitudes,
                                   uccsd_singlet_generator, uccsd_generator,
                                   uccsd_convert_amplitude_format)
from itertools import combinations

In [2]:
with open('data/molecule_geometries/molecule_data.json') as jfile:
    molecule_geometries = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3G_SINGLET', 'CHONH2_STO-3G_SINGLET'])


In [6]:
# Set molecule parameters
speciesname = 'CH3OH_STO-3G_SINGLET'
mol_data = molecule_geometries[speciesname]

atoms = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
multiplicity = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))

delete_input = True
delete_output = True
cisd=0
ccsd=0
fci =1 # wouldn't advise turning this on over 32 qubits!

# Run pyscf.
molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
calculated_molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=cisd,run_ccsd=ccsd,run_fci=fci)

In [7]:
oh = ordering_heuristics(calculated_molecule)

------------------------------------------------
Information concerning the full system:
------------------------------------------------
Number of qubits in full problem: 28
The Hartree-Fock state is |1111111111111111110000000000>
Hartree-Fock energy   = -113.54919325
Møller–Plesset energy = -113.63932094
FCI energy            = -113.67203929
------------------------------------------------
Tapering information:
------------------------------------------------
We are able to taper 3 qubits from the Hamiltonian
The symmetry sector is [-1, -1, 1]
The tapered Hartree-Fock state is |1111111111111110000000000>
------------------------------------------------
CS-VQE information:
------------------------------------------------
Noncontextual GS energy: -113.54919325240562
Symmetry generators:     ['ZIZIZIZIZZIZIIZZIZIIZZIZI', 'IZIZIZIZZIZIZIZIZIZIZIZIZ', 'IIZIZIZIZZIZIIZZIZIIZZIZI', 'IIIZIZIZZIZIZIZIZIZIZIZIZ', 'IIIIZIZIZZIZIIZZIZIIZZIZI', 'IIIIIZIZZIZIZIZIZIZIZIZIZ', 'IIIIIIZIZZIZIIZZIZIIZZ

In [8]:
methods = ["a", "b", "c", "d"]#, "e", "f"]
data = {"methods":methods}
for m in methods:
    print(m)
    data[m] = oh.heuristic_errors(heuristic=m, print_info=True)

data['num_tapered']= oh.n_taper
data['num_qubits'] = oh.n_qubits
data['stabilizers']= oh.generators
data['hf_energy']  = oh.hf_energy
data['mp_energy']  = oh.mp_energy
data['fci_energy'] = oh.fci_energy

with open('data/cs_vqe_results/'+speciesname+'_heuristic_errors.json', 'w') as outfile:
    json.dump(data, outfile)

a
Number of qubits simulated: 1
CS-VQE error w.r.t. HF energy:  0.0000000024
CS-VQE error w.r.t. MP2 energy: 0.0901276912
CS-VQE error w.r.t. FCI energy: 0.1228460426

Number of qubits simulated: 2
CS-VQE error w.r.t. HF energy: -0.0009041520
CS-VQE error w.r.t. MP2 energy: 0.0892235368
CS-VQE error w.r.t. FCI energy: 0.1219418882

Number of qubits simulated: 3
CS-VQE error w.r.t. HF energy: -0.0034787335
CS-VQE error w.r.t. MP2 energy: 0.0866489552
CS-VQE error w.r.t. FCI energy: 0.1193673067

Number of qubits simulated: 4
CS-VQE error w.r.t. HF energy: -0.0035104123
CS-VQE error w.r.t. MP2 energy: 0.0866172765
CS-VQE error w.r.t. FCI energy: 0.1193356280

Number of qubits simulated: 5
CS-VQE error w.r.t. HF energy: -0.0035617463
CS-VQE error w.r.t. MP2 energy: 0.0865659424
CS-VQE error w.r.t. FCI energy: 0.1192842939

Number of qubits simulated: 6
CS-VQE error w.r.t. HF energy: -0.0045695045
CS-VQE error w.r.t. MP2 energy: 0.0855581843
CS-VQE error w.r.t. FCI energy: 0.1182765358

Nu

Number of qubits simulated: 8
CS-VQE error w.r.t. HF energy: -0.0009041620
CS-VQE error w.r.t. MP2 energy: 0.0892235268
CS-VQE error w.r.t. FCI energy: 0.1219418783

Number of qubits simulated: 9
CS-VQE error w.r.t. HF energy: -0.0009041619
CS-VQE error w.r.t. MP2 energy: 0.0892235268
CS-VQE error w.r.t. FCI energy: 0.1219418783

Number of qubits simulated: 10
CS-VQE error w.r.t. HF energy: -0.0009041519
CS-VQE error w.r.t. MP2 energy: 0.0892235368
CS-VQE error w.r.t. FCI energy: 0.1219418883

Number of qubits simulated: 11
CS-VQE error w.r.t. HF energy: -0.0009041619
CS-VQE error w.r.t. MP2 energy: 0.0892235268
CS-VQE error w.r.t. FCI energy: 0.1219418783

Number of qubits simulated: 12
CS-VQE error w.r.t. HF energy: -0.0009041519
CS-VQE error w.r.t. MP2 energy: 0.0892235368
CS-VQE error w.r.t. FCI energy: 0.1219418883

Number of qubits simulated: 13
CS-VQE error w.r.t. HF energy: -0.0010942660
CS-VQE error w.r.t. MP2 energy: 0.0890334227
CS-VQE error w.r.t. FCI energy: 0.1217517742

